## Quantifying image quality (Part 1/2)
This is a partly manual method for looking at the contrast in an image. Images were acquired with different combinations of camera settings and this code examines which one results in better quality (solely in terms of snr).  
Summary : Each bead in an image is detected and a 15x15 pixel roi is drawn around it to find measures like total number of photons in the signal, average photons per pixel, etc. This along with background info is used to calculate signal-to-background. Since the definition of signal-to-background varies with context, it's calculated here in 4 different ways.  
More details can be found in report #16 (mostly after update #3)  
Concerned data-folder : session16  


In [ ]:
import os
import numpy
import glob
import pandas as pd

import storm_analysis.sa_library.datareader as datareader

# Change directory
os.chdir("D:/gayatri-folder/session16/ROI3")

# Load images
# Is the full path required since I changed directory?
images = sorted(glob.glob('D:/gayatri-folder/session16/ROI3/*.tif'))

In [ ]:
# Define functions for peak finding and background calculation

def findPeaks(image_ph, threshold):

    peaks = []
    bg = []
    roi_size = 15 # Size of roi used for peak finding
    mid = (roi_size+1)*(roi_size-1)/2
    offset = ((roi_size-1)/2)
    for x in range(256-(roi_size-1)) :
                for y in range(256-(roi_size-1)) :
                    neighbours=[]
                    for i in range(roi_size) :
                        for j in range(roi_size) :
                            neighbours.append(image_ph[x+i][y+j])
                    maxpos = neighbours.index(max(neighbours))

                    # Save detected peaks.
                    if maxpos == mid and max(neighbours)>=threshold*2:
                        # print(neighbours)
                        peaks.append([x+offset,y+offset,y+offset,x+offset,max(neighbours),numpy.around(numpy.mean(neighbours), decimals=3), numpy.sum(neighbours)])
    df = pd.DataFrame(peaks, columns=['row','col','x','y','peak','avg', 'sum'])

# Manually identify regions in the image which are definitely background.
# These regions are specific to tyhe images in a folder
# For ROI 1 !
    # for x,y in [[30,40], [90,195], [124,110], [195,45], [200,200]] :
# For ROI2 !
    # for x,y in [[95,40], [178,56], [150,120], [79,185], [153,188]] :
# For ROI 3 !
    for x,y in [[38,40], [36,100], [73,168], [138,56], [210,179]] :
                neighbours=[]
                for i in range(roi_size) :
                    for j in range(roi_size) :
                        neighbours.append(image_ph[x+i][y+j])
                # print(neighbours)
                bg.append([numpy.around(numpy.mean(neighbours), decimals=3), numpy.sum(neighbours)])
    bgr = pd.DataFrame(bg, columns=['avg', 'sum'])
    return df, bgr

def findBackground(peaks, frame):
    mask = numpy.ma.make_mask(frame,copy=True,shrink=True,dtype=numpy.bool)
    mask[:,:] = False
    for i,j in peaks.iterrows():
        mask[(int(j[0])-7):(int(j[0])+7), (int(j[1])-7):(int(j[1])+7)]=True
    masked_image = numpy.ma.masked_array(frame, mask=mask)
    background = numpy.ma.mean(masked_image)
    return background, masked_image

In [ ]:
# Convert counts to photons
for img in images :
    image = datareader.TifReader(img)
    image = image.loadAFrame(0)
    img_name = img.rsplit("\\",1)[1][:].rsplit('.tif',1)[0][:]
    print(img_name)
    settings = {'emgain_0001':[1,4.74,200], 'emgain_0002':[20,4.74,100], 'emgain_0003':[40,4.74,100], 'emgain_0004':[60,4.74,100], 'emgain_0005':[80,4.74,100], 'emgain_0006':[100,4.74,100], 'emgain_extra_0030':[30,4.74,100], 'emgain_extra_0065':[65,4.74,100], 'preamp_0001':[65, 16.2, 100], 'preamp_0002':[65, 8.14, 100], 'preamp_0003':[65, 4.74, 100], 'vsspeed_0002':[65,4.74,100], 'vsspeed_0003':[65,4.74,100], 'vsspeed_0004':[65,4.74,100], 'vsspeed_0005':[65,4.74,100]}
    
    if img_name in settings:
        em_gain = settings[img_name][0]
        preamp_gain = settings[img_name][1]
        threshold = settings[img_name][2]

    bias = 500.0
    qe = 0.9
    image_ph = numpy.around(numpy.where(image >= bias, ((image-bias)*preamp_gain)/(em_gain*qe), 1))
    peaks, bgr = findPeaks(image_ph, threshold=threshold)
    peaks_counts, bgr_counts = findPeaks(image, threshold=450)
    # bg, masked_image = findBackground(peaks, image_ph)
    background_sum = bgr['sum'].mean()
    background_avg = bgr['avg'].mean()
    background_sum_counts = bgr_counts['sum'].mean()
    background_avg_counts = bgr_counts['avg'].mean()
    print('Background = ', numpy.around(bgr['avg'].mean(), decimals = 1), ' photons')
    peaks['snr1'] = numpy.around(peaks['sum'].div(background_sum), decimals=3)
    peaks['snr2'] = numpy.around(peaks['sum'].div(background_avg), decimals=3)
    peaks['snr3'] = numpy.around(peaks['avg'].div(background_avg), decimals=3)
    peaks['snr4'] = numpy.around(peaks_counts['avg'].div(background_avg_counts), decimals=3)
    peaks.to_csv("peaks_"+img_name+".csv", header=['row','col','x','y','peak_ph','avg_ph', 'sum_ph', 'snr1', 'snr2', 'snr3', 'snr4'], index=False)
    # peaks_counts.to_csv("counts_"+img_name+".csv", header=['row','col','x','y','peak','avg', 'sum'], index=False)

In [ ]:
os.chdir("D:/gayatri-folder/session16/ROI3")

# Load csv files
csv_files = sorted(glob.glob('D:/gayatri-folder/session16/ROI3/peaks_*.csv'))

In [ ]:
dataframes = []
for file in csv_files:
    df = pd.read_csv(file) # additional arguments up to your need
    df['exp'] = file.rsplit("\\",1)[1][:].rsplit('.csv',1)[0][:]
    dataframes.append(df)
all_of_em = pd.concat(dataframes)

In [ ]:
all_of_em['exp'] = all_of_em['exp'].replace(['peaks_emgain_0001', 'peaks_emgain_0002', 'peaks_emgain_0003', 'peaks_emgain_0004', 'peaks_emgain_0005', 'peaks_emgain_0006', 'peaks_emgain_extra_0030', 'peaks_emgain_extra_0065', 'peaks_preamp_0001', 'peaks_preamp_0002', 'peaks_preamp_0003', 'peaks_vsspeed_0002', 'peaks_vsspeed_0003','peaks_vsspeed_0004', 'peaks_vsspeed_0005'], ['EM Gain 1', 'EM Gain 20', 'EM Gain 40', 'EM Gain 60', 'EM Gain 80', 'EM Gain 100', 'EM Gain 30', 'EM Gain 65', 'Preamp 1', 'Preamp 2', 'Preamp 3', 'vsspeed 0.5', 'vsspeed 0.7','vsspeed 1.5', 'vsspeed 3.3'])
all_of_em.to_csv("all.csv", header=['row','col','x','y','peak_ph','avg_ph', 'sum_ph', 'snr1', 'snr2', 'snr3', 'snr4', 'exp'],index=False)